![imagen](./img/hundir-la-flota-juego-de-mesa.jpg)

En esta entrega vas a crear tu propio juego de **Hundir la flota** en Python. 
[Aquí](http://es.battleship-game.org/) podrás probarlo online.

### En clase
1. Crea la función `crear_tablero(tamaño)`, un tablero por defecto de 10x10 relleno del carácter "_" con numpy.
2. Crea la función `colocar_barco(barco, tablero)`, que recibirá la lista de casillas de un barco y el tablero donde colocarlo. Prueba primero a posicionar un par de barcos por ejemplo en [(0,1), (1,1)] y [(1,3), (1,4), (1,5), (1,6)]. Los barcos serán Os mayúsculas. Como ves, un barco de dos posiciones de eslora y otro de cuatro.
3. Crea la función `disparar(casilla, tablero)`, si el disparo acierta en un barco sustituye la O por una X (tocado), si es agua, sustituye la _ por una A (Agua). Prueba primero a disparar el barco de 2 casillas.
4. Crea la función `crear_barco(eslora)`, que deberá crear una lista de casillas de un barco en función a la eslora, de forma aleatoria.

### Proyecto individual
1. Crea la función `colocar_barcos(tablero)`, que deberá de colocar la lista de barcos generados de forma aleatoria (6 barcos en total (3 barcos de eslora 2, 2 de eslora 3 y 1 eslora 4)) ¡Mucho ojo con barcos que estén superpuestos (no pueden ocupar dos barcos la misma casilla) o barcos que se salgan del tablero!
2. Escribe el flujo completo del programa, con la dinámica de turnos y funcionalidades necesarios para jugar contra la máquina (dispara a tu tablero de forma aleatoria). Crea todas las funciones que necesites y aplica todo lo aprendido que te sea útil.
3. Encapsula todo en un `main.py` y un `utils.py` para ejecutarlo desde terminal.
4. Sube tu proyecto a un repositorio de github y prepara una demo (solo se podrá enseñar desde terminal) para la presentación de tu proyecto.

## Presentación
Cada uno realizará una presentación el **lunes 17 de marzo**, donde se contarán con **10 minutos máximo**, importante ceñirse al tiempo. Se tendrá que enseñar:
1. El git clone del repositorio de github a tu ordenador y explicar las partes más relevantes del código.
2. Una demo donde se muestre el correcto funcionamiento del código para jugar, ejecutándose desde terminal.
3. Explicación de la lógica de las partes más relevantes del código desarrollado.

In [ ]:
# !pip install numpy

In [ ]:
# Traremos numpy para poder trabajar con arreglos numericos o matrices

import numpy as np
import random
import os
import time

In [ ]:
#Creamos la clase tablero, con un parametro fijo de tamano (10, 10) y le agregamos un metodo crear_tablero

class Tablero:
    def __init__(self, tamano=(10, 10)):
        self.tamano = tamano
        self.grid = np.full(tamano, '_')
        self.tablero_disparos = np.full(tamano, '_')  # Agregamos un tablero de disparos contra el enemigo
        self.disparos_realizados = []  # Lista para almacenar las casillas disparadas
        self.disparos_recibidos = []   # Nueva lista para disparos recibidos

    def mostrar_tablero(self):
        print("Sus barcos capitán: ")
        print(self.grid)
        print("\nTu tablero de disparos contra su enemigo: ")
        print(self.tablero_disparos)

    def recibir_disparo(self, casilla):
        #Proceso disparos recibidos (usado por la computadora contra el jugador)
        fila, columna = casilla
        if fila >= self.tamano[0] or columna >= self.tamano[1]:
            return "Disparo fuera del tablero" # Se devuelve la respuesta como una cadena anidada, asi se muestran abajo del tablero
        if casilla in self.disparos_recibidos:  # Cambié a disparos_recibidos
            return "La computadora ya disparó aquí"
        self.disparos_recibidos.append(casilla)
        if self.grid[fila, columna] == 'O':
            self.grid[fila, columna] = 'X'
            return "¡La computadora ha acertado y comprometido una de tus flotas!"
        elif self.grid[fila, columna] == '_':
            self.grid[fila, columna] = 'A'
            return "La computadora ha fallado"
        return ""
    
    def disparar(self, casilla, tablero_enemigo):
        """
        Registra el disparo del jugador en el tablero de disparos 
        y actualiza el tablero enemigo
        """
        fila, columna = casilla
        if casilla in self.disparos_realizados:
            return "Ya disparaste aquí"
        self.disparos_realizados.append(casilla) # Si llega hasta aqui es porque la casilla no esta en el conjunto de las previas
        if tablero_enemigo.grid[fila, columna] == 'O':
            self.tablero_disparos[fila, columna] = 'X'
            tablero_enemigo.grid[fila, columna] = 'X'  # Actualiza el tablero enemigo
            return "¡Acertaste!"
        else:
            self.tablero_disparos[fila, columna] = 'A'
            tablero_enemigo.grid[fila, columna] = 'A'
            return "Has fallado al enemigo"
        return ""


In [20]:
# class Barco

class Barco:
    def __init__(self, posiciones):
        self.posiciones = posiciones

    def colocar_barco(self, tablero):
        for fila, columna in self.posiciones:
        # Eliminamos el if, y dejamos solo la colocacion del barco, asumiendo que las posiciones ya son validas
            tablero.grid[fila, columna] = 'O'
        return tablero
    
    @staticmethod
    def crear_barco_aleatorio(eslora, tablero):
        while True:
            orientacion = random.choice(["Horizontal", "Vertical"])
            if orientacion == "Horizontal":
                fila = random.randint(0, tablero.tamano[0] - 1)
                columna_inicial = random.randint(0, tablero.tamano[1] - eslora)
            else:
                fila_inicial = random.randint(0, tablero.tamano[0] - eslora)
                columna = random.randint(0, tablero.tamano[1] - 1)

            barco_aleatorio = []
            for i in range(eslora):
                if orientacion == "Horizontal":
                    casilla = (fila, columna_inicial + i)
                else:
                    casilla = (fila_inicial + i, columna)
                
                # Verificar antes de añadir
                fila_casilla, columna_casilla = casilla
                if tablero.grid[fila_casilla, columna_casilla] != '_':
                    break  # Si está ocupada, salimos del for y probamos otra vez
                barco_aleatorio.append(casilla)
            else:  # Se ejecuta si el for termina sin break
                return barco_aleatorio  # Todas las posiciones estaban libres

In [21]:
# Funcion independeinte para colocar una flota aleatoria en un tablero

def colocar_flota_aleatoria(tablero, flota):
    for eslora, cantidad in flota:
        for i in range(cantidad):
            barco = Barco(Barco.crear_barco_aleatorio(eslora, tablero))
            tablero = barco.colocar_barco(tablero)
    return tablero

# Función para limpiar la consola
def limpiar_consola():
    os.system('cls' if os.name == 'nt' else 'clear')

def calcular_ganador(tablero_jugador, tablero_computadora):
    """Calcula los porcentajes de aciertos y determina el ganador"""
    # Aciertos del jugador (contar 'X' en el tablero de la computadora)
    aciertos_jugador = np.sum(tablero_computadora.grid == 'X')
    disparos_jugador = len(tablero_jugador.disparos_realizados)
    porcentaje_jugador = (aciertos_jugador / disparos_jugador * 100) if disparos_jugador > 0 else 0

    # Aciertos de la computadora (contar 'X' en el tablero del jugador)
    aciertos_computadora = np.sum(tablero_jugador.grid == 'X')
    disparos_computadora = len(tablero_jugador.disparos_recibidos)
    porcentaje_computadora = (aciertos_computadora / disparos_computadora * 100) if disparos_computadora > 0 else 0

    print(f"\nEstadísticas finales:")
    print(f"Jugador - Aciertos: {aciertos_jugador}/{disparos_jugador} ({porcentaje_jugador:.2f}%)")
    print(f"Computadora - Aciertos: {aciertos_computadora}/{disparos_computadora} ({porcentaje_computadora:.2f}%)")

    if porcentaje_jugador > porcentaje_computadora:
        print(f"¡Ganaste, {nombre_jugador}! Tu precisión fue mayor.")
    elif porcentaje_computadora > porcentaje_jugador:
        print("La computadora gana. Su precisión fue mayor.")
    else:
        print("¡Empate! Ambos tuvieron la misma precisión.")

In [22]:
# Flota estándar para el juego
flota = [
    (4, 1),  # 1 barco de eslora 4
    (3, 2),  # 2 barcos de eslora 3
    (2, 3),  # 3 barcos de eslora 2
]

# Flujo inicial del programa
print("¡Bienvenido a Hundir la Flota Capitán!")
nombre_jugador = input("Por favor, ingrese su nombre: ")
print(f"¡Hola, {nombre_jugador}! Preparando los tableros...")

# Crear tableros
tablero_jugador = Tablero()  # Visible para el jugador
tablero_computadora = Tablero()  # No visible para el jugador

# Colocar barcos aleatorios en ambos tableros
tablero_jugador = colocar_flota_aleatoria(tablero_jugador, flota)
tablero_computadora = colocar_flota_aleatoria(tablero_computadora, flota)

# Limpiar para volver al estado inicial y mostrar solo el tablero del jugador
limpiar_consola()
print(f"Estado del juego, {nombre_jugador}:")
tablero_jugador.mostrar_tablero()
print("\n¡Comienza el juego! Tu turno primero.")
print("Ingresa las coordenadas para disparar (fila columna, ej: 0 0) o 'salir' para terminar:")

# Juego con turnos
max_turnos = 10
turno = 0
mensaje_jugador = ""
mensaje_computadora = ""

while turno < max_turnos:
    # Turno del jugador
    try:
        entrada = input().strip().lower()
        if entrada == 'salir':
            break
        fila, columna = map(int, entrada.split())
        mensaje_jugador = tablero_jugador.disparar((fila, columna), tablero_computadora)
    except (ValueError, IndexError):
        mensaje_jugador = "Entrada inválida. Usa dos números entre 0 y 9 separados por un espacio (ej: 0 0)."

    # Turno de la computadora
    fila = random.randint(0, 9)
    columna = random.randint(0, 9)
    mensaje_computadora = tablero_computadora.recibir_disparo((fila, columna))
    mensaje_computadora += f"\nLa computadora disparó a ({fila}, {columna})"

    # Actualizar pantalla
    limpiar_consola()
    print(f"Estado del juego, {nombre_jugador} (Turno {turno + 1}):")
    tablero_jugador.mostrar_tablero()
    print(f"\n{mensaje_jugador}")
    print(f"{mensaje_computadora}")
    print("\nIngresa las coordenadas para disparar (fila columna, ej: 0 0) o 'salir' para terminar:")
    
    turno += 1
    time.sleep(1)

# Calcular y mostrar el ganador
limpiar_consola()
print(f"Estado final del juego, {nombre_jugador}:")
tablero_jugador.mostrar_tablero()
calcular_ganador(tablero_jugador, tablero_computadora)


¡Bienvenido a Hundir la Flota Capitán!
¡Hola, ric! Preparando los tableros...
Estado del juego, ric:
Sus barcos capitán: 
[['_' '_' '_' '_' '_' '_' 'O' 'O' 'O' '_']
 ['_' 'O' '_' '_' '_' '_' '_' 'O' 'O' 'O']
 ['_' 'O' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' 'O' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' 'O' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' 'O' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' 'O' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' 'O' 'O' '_' '_' 'O' 'O' '_' '_' '_']]

Tu tablero de disparos contra su enemigo: 
[['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_